In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import panel as pn
import holoviews as hv

import param # Panel's parameter handling
# Assuming necessary visualization libraries are imported
from bokeh.palettes import Viridis256 # Example palette
# Import HoverTool explicitly if needed from bokeh.models import HoverTool

hv.extension('bokeh')
pn.extension()

# --- Define the Visualization Class using Panel's 'param' ---
class HMMExplorer(param.Parameterized):
    # --- Data Parameters (Set these when creating an instance) ---
    posterior_da = param.Parameter(doc="xarray DataArray (time, y, x) of posterior probabilities")
    position_df = param.Parameter(doc="pandas DataFrame (indexed by time) with 'x', 'y' columns for animal position")
    timeseries_df = param.Parameter(doc="pandas DataFrame (indexed by time) with other time series")
    environment = param.Parameter(doc="Environment object with spatial info (edges_, etc.)")

    # --- Widgets / Control Parameters ---
    time_index = param.Integer(default=0, label="Time Index")
    window_seconds = param.Number(1.0, bounds=(0.1, 10.0), step=0.1, label="Time Slice Window (s)")
    play_speed = param.Number(5.0, bounds=(0.1, 50.0), label="Playback Speed (X Realtime)")
    is_playing = param.Boolean(False, label="Play")

    # --- Internal State Variables (Derived from Data) ---
    n_time = param.Integer(default=0, precedence=-1)
    time_step_size = param.Number(default=0.0, precedence=-1)
    time_vector = param.Array(default=np.array([]), precedence=-1)

    # Internal state for playback
    _periodic_callback = None # Stores the callback object returned by add_periodic_callback
    _overview_plots_cache = None
    _overview_vlines = param.Dict({})

    def __init__(self, **params):
        """Initialize and trigger initial data update."""
        self._overview_plots_cache = pn.Column("### Full Time Series\n(Initializing...)")
        super().__init__(**params)
        if self.posterior_da is not None and self.timeseries_df is not None and self.environment is not None and self.position_df is not None:
            self._update_internal_data()
        else:
            print("Warning: Initialize HMMExplorer with posterior_da, position_df, timeseries_df, and environment for full functionality.")

    @param.depends('posterior_da', watch=True, on_init=True)
    def _update_internal_data(self):
        # (Same as previous correct version)
        if not isinstance(self.posterior_da, xr.DataArray):
            self.n_time = 0
            self.time_step_size = 0.0
            self.time_vector = np.array([])
            self.param['time_index'].bounds = (0, 0)
            self.param['time_index'].step = 1
            self.time_index = 0
            self._overview_plots_cache = pn.Column("### Full Time Series\n(Load Posterior Data)")
            if self.is_playing: self.is_playing = False
            return

        _time_vector = self.posterior_da['time'].values
        _n_time = len(_time_vector)
        _time_step_size = np.mean(np.diff(_time_vector)) if _n_time > 1 else 0.0

        self.n_time = _n_time
        self.time_step_size = _time_step_size
        self.time_vector = _time_vector

        if self.n_time > 0:
            self.param['time_index'].bounds = (0, self.n_time - 1)
            self.param['time_index'].step = max(1, self.n_time // 1000)
            if self.time_index >= self.n_time:
                 self.time_index = 0
        else:
             self.param['time_index'].bounds = (0, 0)
             self.param['time_index'].step = 1
             self.time_index = 0

        if self.timeseries_df is not None and isinstance(self.timeseries_df, pd.DataFrame):
            self._create_overview_plots()
        else:
             self._overview_plots_cache = pn.Column("### Full Time Series\n(Load Timeseries Data)")

    # --- Methods to create plot components ---
    def _create_overview_plots(self):
        # (Same as previous correct version, ensure df_to_plot_base uses self.time_vector)
        if self.timeseries_df is None or self.time_vector is None or len(self.time_vector) == 0:
             self._overview_plots_cache = pn.Column("### Full Time Series\n(Load Timeseries/Posterior Data)")
             return

        overview_plots = []
        new_vlines = {}
        current_time_idx = min(self.time_index, len(self.time_vector) - 1) if len(self.time_vector) > 0 else 0
        current_time = self.time_vector[current_time_idx] if len(self.time_vector) > 0 else 0

        df_to_plot_base = self.timeseries_df.copy()
        df_to_plot_base['time_col_for_plot'] = self.time_vector
        time_col_name = 'time_col_for_plot'

        for col in self.timeseries_df.columns:
            if col not in df_to_plot_base.columns: continue

            plot = df_to_plot_base.hvplot(
                 x=time_col_name, y=col, kind='line',
                 width=450, height=100,
                 datashade=True, rasterize=True,
                 xlabel="Time (s)", ylabel=col, title=col, grid=True,
                 xlim=(self.time_vector.min(), self.time_vector.max())
            ).opts(shared_axes=False)

            vline = hv.VLine(current_time).opts(color='red', line_width=1)
            new_vlines[col] = vline
            overview_plots.append(plot * vline)

        self._overview_vlines = new_vlines
        self._overview_plots_cache = pn.Column("### Full Time Series", hv.Layout(overview_plots).cols(1))


    # --- Methods bound to parameters for dynamic updates ---
    # (view_2d_posterior, view_time_slice remain the same)
    @param.depends('time_index')
    def view_2d_posterior(self):
        if not isinstance(self.posterior_da, xr.DataArray) or self.environment is None or not isinstance(self.position_df, pd.DataFrame) or self.n_time == 0:
             return hv.Image([], kdims=['x', 'y']).opts(width=450, height=400, title="Posterior (Load Data)")
        idx = min(self.time_index, self.n_time - 1)
        selected_time = self.time_vector[idx]
        try:
            posterior_slice = self.posterior_da.sel(time=selected_time, method='nearest')
            posterior_map = hv.Image(
                posterior_slice, kdims=['x', 'y'], vdims=[self.posterior_da.name or 'Posterior Probability']
            ).opts(
                cmap=Viridis256, colorbar=True, width=450, height=400, tools=['hover'],
                title=f"Posterior @ {selected_time:.3f}s", clim=(0, np.nanpercentile(self.posterior_da.values, 99.5)),
                xlim=(self.environment.edges_[0].min(), self.environment.edges_[0].max()),
                ylim=(self.environment.edges_[1].min(), self.environment.edges_[1].max()),
                xlabel="X Position", ylabel="Y Position"
            )
        except Exception as e:
             print(f"Error creating posterior map: {e}")
             posterior_map = hv.Image([], kdims=['x', 'y']).opts(width=450, height=400)
        try:
            current_pos = self.position_df.loc[self.position_df.index.asof(selected_time)]
            animal_marker = hv.Scatter((current_pos['x'], current_pos['y'])).opts(marker='x', color='red', size=12, line_width=2)
        except Exception as e:
             animal_marker = hv.Scatter([]).opts(color='red')
        try:
            bounds = hv.Bounds((self.environment.edges_[0].min(), self.environment.edges_[1].min(),
                               self.environment.edges_[0].max(), self.environment.edges_[1].max()))
            return bounds.opts(line_color='gray') * posterior_map * animal_marker
        except Exception as e:
             print(f"Error creating bounds: {e}")
             return posterior_map * animal_marker

    @param.depends('time_index', 'window_seconds')
    def view_time_slice(self):
        if not isinstance(self.timeseries_df, pd.DataFrame) or self.time_vector is None or self.n_time == 0:
            return pn.pane.Markdown("Time Slice (Load Data)")
        idx = min(self.time_index, self.n_time - 1)
        selected_time = self.time_vector[idx]
        window_half_width = self.window_seconds / 2.0
        start_time = selected_time - window_half_width
        end_time = selected_time + window_half_width
        ts_slice_df = self.timeseries_df.loc[start_time:end_time].copy()

        if ts_slice_df.empty:
             layout = hv.Layout([hv.Curve([], label=col).opts(responsive=True, height=100, title=col) for col in self.timeseries_df.columns]).cols(1)
             return layout.opts(title="Time Slice (No Data in Window)")

        ts_slice_df['relative_time'] = ts_slice_df.index - selected_time
        slice_plots = []
        for col in self.timeseries_df.columns:
            # Use explicit tooltips for Bokeh backend
            from bokeh.models import HoverTool as BkHoverTool
            hover = BkHoverTool(tooltips=[('Rel Time', '@relative_time{0.000} s'), (col, f'@{col}{{0.00}}')])
            plot = ts_slice_df.hvplot(
                x='relative_time', y=col, kind='line',
                responsive=True, height=100,
                xlabel="Time rel. to Current (s)", ylabel=col,
                title=col, grid=True,
                tools=[hover] # Pass bokeh hover tool instance
            ).opts(shared_axes=False) * hv.VLine(0).opts(color='red', line_width=1)
            slice_plots.append(plot)
        return hv.Layout(slice_plots).cols(1).opts(title="Time Slice")

    # --- Playback Control Methods ---
    @param.depends('is_playing', watch=True)
    def _play_pause_control(self):
        if self.is_playing:
            if self._periodic_callback is None:
                if self.time_step_size <= 0 or self.n_time <= 1:
                    print("Warning: Cannot play, invalid time parameters.")
                    self.is_playing = False
                    return
                callback_period_ms = max(15, int((self.time_step_size / self.play_speed) * 1000))
                try:
                    # Store the callback object returned by add_periodic_callback
                    self._periodic_callback = pn.state.add_periodic_callback(
                        self._advance_time, period=callback_period_ms
                    )
                except Exception as e:
                    print(f"Error starting periodic callback: {e}")
                    self.is_playing = False
                    self._periodic_callback = None
        else: # If is_playing is False (stop command)
            if self._periodic_callback is not None:
                try:
                    # *** CORRECTED: Call stop() on the callback object ***
                    self._periodic_callback.stop()
                except Exception as e: # More general exception catch
                     print(f"Error stopping periodic callback: {e}")
                     # Fallback: Try removing via pn.state if stop fails (though unlikely needed)
                     try:
                          pn.state.remove_periodic_callback(self._periodic_callback)
                     except Exception:
                          pass
                finally:
                    self._periodic_callback = None # Clear the stored object

    @param.depends('play_speed', watch=True)
    def _speed_control(self):
         # Only adjust if currently playing
         if self.is_playing and self._periodic_callback is not None:
            if self.time_step_size <= 0:
                print("Warning: Cannot adjust speed, invalid time_step_size.")
                self.is_playing = False # Stop playing if step size is bad
                return

            current_period = self._periodic_callback.period
            new_period_ms = max(15, int((self.time_step_size / self.play_speed) * 1000))

            if current_period != new_period_ms:
                try:
                    # *** CORRECTED: Call stop() on the callback object ***
                    self._periodic_callback.stop()
                except Exception as e:
                    print(f"Error stopping periodic callback for speed change: {e}")
                     # Fallback if stop fails
                     try:
                          pn.state.remove_periodic_callback(self._periodic_callback)
                     except Exception:
                          pass
                finally:
                    self._periodic_callback = None # Ensure it's cleared before adding new

                try:
                     # Store the new callback object
                    self._periodic_callback = pn.state.add_periodic_callback(
                        self._advance_time, period=new_period_ms
                    )
                except Exception as e:
                    print(f"Error restarting periodic callback with new speed: {e}")
                    self.is_playing = False
                    self._periodic_callback = None

    def _advance_time(self):
        # Use param trigger_guard to prevent potential infinite loops if updates trigger watchers
        with param.parameterized.discard_events(self):
            if self.time_index < self.n_time - 1:
                self.time_index += 1 # Increment time index
            else:
                self.is_playing = False # Stop at the end (will trigger _play_pause_control)

    # Update overview vlines when time_index changes (manually or via play)
    @param.depends('time_index', watch=True)
    def _update_overview_vlines(self):
        # (Keep this method as before - it was correct)
        if not hasattr(self, '_overview_vlines') or not self._overview_vlines \
           or self.time_vector is None or self.n_time == 0:
             return
        idx = min(self.time_index, self.n_time - 1)
        selected_time = self.time_vector[idx]
        for col, vline in self._overview_vlines.items():
             try:
                  if vline is not None:
                       vline.update(selected_time) # Update HoloViews VLine position
             except Exception as e:
                  pass

    # --- Define Panel Layout ---
    def view(self):
        # (Keep this method as before - it was correct)
        if self._overview_plots_cache is None :
            if self.timeseries_df is not None and self.time_vector is not None and self.n_time > 0:
                 self._create_overview_plots()
            else:
                  overview_panel = pn.Column("### Full Time Series\n(Load Data)")
        if self._overview_plots_cache is not None :
             overview_panel = self._overview_plots_cache
        else:
             overview_panel = pn.Column("### Full Time Series\n(Error creating overview)")

        controls = pn.WidgetBox(
            '### Controls',
            self.param.time_index,
            pn.Row(
                 pn.Param(self.param.is_playing, widgets={'is_playing': {'type': pn.widgets.Toggle, 'button_type': 'success', 'name': 'Play/Pause'}}),
                 self.param.play_speed
            ),
            self.param.window_seconds
        )

        layout = pn.Row(
            pn.Column(
                "### Decoded Posterior Probability",
                self.view_2d_posterior,
                controls
            ),
            pn.Column(
                "### Time Slice View",
                self.view_time_slice,
                overview_panel
            )
        )
        return layout

# --- Example Usage ---
if __name__ == "__main__":
    # 1. Generate the fake data first
    (time_vector_data,
     spatial_posterior_da_data,
     animal_pos_df_data,
     timeseries_df_data,
     environment_data) = generate_fake_data(duration_minutes=1, time_bin_ms=4)

    print("Fake data generated. Instantiating HMMExplorer...")

    # 2. Instantiate the class with the generated data
    explorer = HMMExplorer(name="Replay Explorer",
                           posterior_da=spatial_posterior_da_data,
                           position_df=animal_pos_df_data,
                           timeseries_df=timeseries_df_data,
                           environment=environment_data)

    print("Explorer instantiated. Displaying view...")

    # 3. Display the view in Jupyter
    from IPython.display import display
    display(explorer.view())

    print("View displayed. Interact with the controls.")

IndentationError: unexpected indent (3175403422.py, line 238)

In [36]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt  # Only for verification plot if needed
from scipy.ndimage import gaussian_filter1d
from dataclasses import dataclass, field


# --- Define a simple mock Environment class ---
# (Replicating essential parts from environment.py for standalone example)
@dataclass
class MockEnvironment:
    environment_name: str = "MockEnv"
    place_bin_size: float = 5.0
    position_range: tuple = field(
        default_factory=lambda: ((0, 100), (0, 100))
    )  # Example 100x100 cm box
    edges_: tuple = field(init=False)
    place_bin_centers_: np.ndarray = field(init=False)
    centers_shape_: tuple = field(init=False)
    is_track_interior_: np.ndarray = field(init=False)

    def __post_init__(self):
        # Simplified grid creation
        self.edges_ = tuple(
            np.arange(start, stop + self.place_bin_size, self.place_bin_size)
            for start, stop in self.position_range
        )
        centers_x = self.edges_[0][:-1] + np.diff(self.edges_[0]) / 2
        centers_y = self.edges_[1][:-1] + np.diff(self.edges_[1]) / 2
        self.centers_shape_ = (
            len(centers_y),
            len(centers_x),
        )  # Note: (n_y_bins, n_x_bins)
        xv, yv = np.meshgrid(centers_x, centers_y)
        self.place_bin_centers_ = np.vstack([xv.ravel(), yv.ravel()]).T
        # Assume all bins are interior for simplicity
        self.is_track_interior_ = np.ones(self.centers_shape_, dtype=bool)

    def find_bin(self, position):
        # Find bin index for a given position (simplified)
        pos = np.asarray(position)
        bin_x = np.digitize(pos[..., 0], self.edges_[0]) - 1
        bin_y = np.digitize(pos[..., 1], self.edges_[1]) - 1
        # Clip to valid bin indices
        bin_x = np.clip(bin_x, 0, self.centers_shape_[1] - 1)
        bin_y = np.clip(bin_y, 0, self.centers_shape_[0] - 1)
        # Ravel multi index requires indices corresponding to dimensions (y, x)
        return np.ravel_multi_index((bin_y, bin_x), self.centers_shape_)


def generate_fake_data(duration_minutes=5, time_bin_ms=2):
    """Generates fake data for testing the HMM Explorer visualization."""

    print("Generating fake data...")
    # --- Time Vector ---
    fs = 1000 / time_bin_ms  # Sampling frequency in Hz
    total_seconds = duration_minutes * 60
    n_time = int(total_seconds * fs)
    time_vector = np.arange(n_time) / fs
    print(
        f"Generated time vector: {n_time} points, {total_seconds:.1f} s total, {1/fs:.4f} s step"
    )

    # --- Environment ---
    env = MockEnvironment(place_bin_size=5.0, position_range=((0, 100), (0, 100)))
    n_xbins = env.centers_shape_[1]
    n_ybins = env.centers_shape_[0]
    n_pos_bins = n_xbins * n_ybins
    pos_bin_centers_2d = env.place_bin_centers_.reshape(n_ybins, n_xbins, 2)
    print(f"Generated environment: {n_xbins} x {n_ybins} = {n_pos_bins} bins")

    # --- Animal Position ---
    # Simulate a smoothed random walk
    pos_std = 0.8  # Std deviation of step size
    pos_smooth_sigma_sec = 0.5  # Smoothing factor in seconds
    position = np.random.randn(n_time, 2) * pos_std
    position[0, :] = [50, 50]  # Start in center
    position = np.cumsum(position, axis=0)
    # Smooth the position trajectory
    position = gaussian_filter1d(position, sigma=pos_smooth_sigma_sec * fs, axis=0)
    # Keep within bounds
    position[:, 0] = np.clip(
        position[:, 0], env.position_range[0][0], env.position_range[0][1]
    )
    position[:, 1] = np.clip(
        position[:, 1], env.position_range[1][0], env.position_range[1][1]
    )

    animal_pos_df = pd.DataFrame(position, index=time_vector, columns=["x", "y"])
    print("Generated animal position data.")

    # --- Other Time Series ---
    # Speed (derived from position, with noise)
    speed = (
        np.sqrt(np.sum(np.diff(position, axis=0, prepend=position[[0]]) ** 2, axis=1))
        * fs
    )
    speed = (
        gaussian_filter1d(speed, sigma=0.2 * fs) + np.random.rand(n_time) * 2
    )  # Add noise
    speed = np.clip(speed, 0, None)

    # Dopamine (e.g., slow oscillation + faster noise/events)
    dopamine_slow = np.sin(2 * np.pi * 0.01 * time_vector) * 0.5  # Slow fluctuation
    dopamine_noise = gaussian_filter1d(np.random.randn(n_time), sigma=0.1 * fs) * 0.2
    dopamine_events = (
        np.random.poisson(0.001, size=n_time) * np.random.rand(n_time) * 3
    )  # Sparse positive events
    dopamine = (
        dopamine_slow
        + dopamine_noise
        + gaussian_filter1d(dopamine_events, sigma=0.5 * fs)
        + 1.0
    )  # Add baseline

    # LFP Theta Power (e.g., higher during movement)
    theta_power = np.clip(speed / 20, 0, 1)  # Base power related to speed
    theta_power += np.random.rand(n_time) * 0.3  # Add noise
    theta_power = gaussian_filter1d(theta_power, sigma=1.0 * fs) * 2.0 + 0.5

    timeseries_df = pd.DataFrame(
        {"Speed": speed, "Dopamine": dopamine, "Theta Power": theta_power},
        index=time_vector,
    )
    print("Generated auxiliary time series data.")

    # --- Spatial Posterior ---
    # Simulate a simple true state (Local vs NonLocal)
    true_state = np.zeros(n_time, dtype=int)  # 0: Local, 1: NonLocal
    # Add some non-local periods (e.g., when speed is low)
    low_speed_threshold = 5.0
    non_local_prob = 0.05  # Base probability
    true_state[speed < low_speed_threshold] = (
        np.random.rand(np.sum(speed < low_speed_threshold)) < non_local_prob
    )

    # Make non-local states persist a bit
    for _ in range(3):  # Apply smoothing/persistence filter
        change_to_nl = (true_state[:-1] == 0) & (true_state[1:] == 1)
        change_to_l = (true_state[:-1] == 1) & (true_state[1:] == 0)
        true_state[1:][change_to_nl] = (
            np.random.rand(change_to_nl.sum()) < 0.8
        )  # Less likely to switch TO NL
        true_state[1:][change_to_l] = (
            np.random.rand(change_to_l.sum()) < 0.1
        )  # More likely to stay NL

    # Simulate posterior based on true state
    # This is a simplified approximation! Real posteriors are smoother.
    local_sigma = 10.0  # Std dev for local state uncertainty (in cm)
    nonlocal_sigma = 25.0  # Std dev for non-local state spread

    # Simulate a moving "replay" focus during non-local periods
    replay_focus = np.zeros_like(position)
    replay_focus[0] = [20, 20]
    replay_walk = gaussian_filter1d(
        np.random.randn(n_time, 2) * 3.0, sigma=1.0 * fs, axis=0
    )
    replay_focus = np.cumsum(replay_walk, axis=0) + replay_focus[0]
    replay_focus[:, 0] = np.clip(
        replay_focus[:, 0], env.position_range[0][0], env.position_range[0][1]
    )
    replay_focus[:, 1] = np.clip(
        replay_focus[:, 1], env.position_range[1][0], env.position_range[1][1]
    )

    # Precompute distances for efficiency (only needed once)
    # Shape: (n_ybins, n_xbins, 2) - (1, 1, 2) -> (n_ybins, n_xbins, 2)
    # Then sum over last axis -> (n_ybins, n_xbins)
    coords_y = env.place_bin_centers_[:, 1].reshape(n_ybins, n_xbins)
    coords_x = env.place_bin_centers_[:, 0].reshape(n_ybins, n_xbins)

    posterior_array = np.zeros((n_time, n_ybins, n_xbins))

    # Batch processing for speed
    batch_size = 1000
    for t_start in range(0, n_time, batch_size):
        t_end = min(t_start + batch_size, n_time)
        print(f"\rGenerating posterior {t_start}-{t_end}/{n_time}", end="")

        # Get centers for the batch
        centers_local = position[t_start:t_end]
        centers_nonlocal = replay_focus[t_start:t_end]
        is_nonlocal_batch = true_state[t_start:t_end] == 1

        # Calculate squared distances in batch
        # Shape: (batch, n_ybins, n_xbins, 2)
        delta_local = (
            env.place_bin_centers_.reshape(1, n_ybins, n_xbins, 2)
            - centers_local[:, np.newaxis, np.newaxis, :]
        )
        delta_nonlocal = (
            env.place_bin_centers_.reshape(1, n_ybins, n_xbins, 2)
            - centers_nonlocal[:, np.newaxis, np.newaxis, :]
        )

        sq_dist_local = np.sum(delta_local**2, axis=-1)  # (batch, n_ybins, n_xbins)
        sq_dist_nonlocal = np.sum(delta_nonlocal**2, axis=-1)

        # Calculate Gaussian likelihoods (unnormalized)
        like_local = np.exp(-0.5 * sq_dist_local / (local_sigma**2))
        like_nonlocal = np.exp(-0.5 * sq_dist_nonlocal / (nonlocal_sigma**2))

        # Assign based on true state for this batch
        batch_posterior = np.where(
            is_nonlocal_batch[:, np.newaxis, np.newaxis], like_nonlocal, like_local
        )

        # Normalize each time slice
        norm = batch_posterior.sum(axis=(1, 2), keepdims=True)
        norm[norm == 0] = 1.0  # Avoid division by zero
        posterior_array[t_start:t_end] = batch_posterior / norm

    print("\nGenerated posterior probability data.")

    # Create xarray DataArray
    spatial_posterior_da = xr.DataArray(
        posterior_array,
        coords={
            "time": time_vector,
            "y_position": coords_y[:, 0],  # y-coordinates of bin centers
            "x_position": coords_x[0, :],  # x-coordinates of bin centers
        },
        dims=["time", "y_position", "x_position"],
        name="Posterior Probability",
    )
    print("Created xarray DataArray.")

    return time_vector, spatial_posterior_da, animal_pos_df, timeseries_df, env

In [ ]:

# 1. Generate the fake data first
(
    time_vector_data,
    spatial_posterior_da_data,
    animal_pos_df_data,
    timeseries_df_data,
    environment_data,
) = generate_fake_data(
    duration_minutes=1, time_bin_ms=4
)  # Shorter duration, larger bins for quicker test

# 2. Instantiate the class with the generated data
explorer = HMMExplorer(
    name="Replay Explorer",
    posterior_da=spatial_posterior_da_data,
    position_df=animal_pos_df_data,
    timeseries_df=timeseries_df_data,
    environment=environment_data,
)

# 3. Display the view in Jupyter
explorer.view().servable()  # Or just display explorer.view()

Generating fake data...
Generated time vector: 15000 points, 60.0 s total, 0.0040 s step
Generated environment: 20 x 20 = 400 bins
Generated animal position data.
Generated auxiliary time series data.
Generating posterior 14000-15000/15000
Generated posterior probability data.
Created xarray DataArray.


Invoked as dynamic_operation(height=100, scale=1.0, width=450, x_range=(np.float64(0.0), np.float64(59.996)), y_range=None)
Invoked as dynamic_operation(height=100, scale=1.0, width=450, x_range=(np.float64(0.0), np.float64(59.996)), y_range=None)
Invoked as dynamic_operation(height=100, scale=1.0, width=450, x_range=(np.float64(0.0), np.float64(59.996)), y_range=None)


AssertionError: <code object compute_scale_and_translate at 0x17b7156b0, file "/Users/edeno/miniconda3/envs/non_local_detector2/lib/python3.12/site-packages/datashader/core.py", line 58> != <code object mapper at 0x18053e250, file "/Users/edeno/miniconda3/envs/non_local_detector2/lib/python3.12/site-packages/datashader/core.py", line 112>

Row
    [0] Column
        [0] Markdown(str)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [2] WidgetBox
            [0] Markdown(str)
            [1] IntSlider(end=14999, name='Time Index', step=15)
            [2] Row
                [0] Param(HMMExplorer, name='Replay Explorer', parameters=['is_playing'], show_name=False, widgets={'is_playing': {'type': <c...})
                [1] FloatSlider(end=50.0, name='Playback Speed (..., start=0.1, value=5.0)
            [3] FloatSlider(end=10.0, name='Time Slice Window (s)', start=0.1, value=1.0)
    [1] Column
        [0] Markdown(str)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [2] Column
            [0] Markdown(str)
            [1] HoloViews(Layout)

In [27]:
import numpy as np
import pandas as pd
import xarray as xr  # Assuming posterior is xarray
import panel as pn
import holoviews as hv
import hvplot.pandas  # Required for pandas plotting
from holoviews.operation.datashader import datashade, rasterize
from bokeh.palettes import Viridis256  # Example palette

# Ensure HoloViews is initialized with the Bokeh extension
hv.extension("bokeh")
# Ensure Panel loads necessary extensions
pn.extension()


# --- Helper: Player Creation (Modified slightly for clarity) ---
def create_player(n_time, sampling_frequency=250, name="Time Index", **kwargs):
    base_interval = int(1000 / sampling_frequency)  # ms per time step
    player = pn.widgets.Player(
        name=name,
        start=0,
        end=n_time - 1,
        value=0,
        step=1,
        interval=base_interval,  # Default to 1x speed
        show_loop_controls=False,
        **kwargs,  # Pass other kwargs like width
    )
    return player


# --- Helper: Speed Control (Unchanged) ---
def create_playback_speed_dropdown(player, sampling_frequency=250, **kwargs):
    speed_options = {
        "1/8x": 8,
        "1/4x": 4,
        "1/2x": 2,
        "1x": 1,
        "2x": 0.5,
        "4x": 0.25,
        "8x": 0.125,
    }
    speed_dropdown = pn.widgets.Select(
        name="Playback Speed", options=speed_options, value=1, **kwargs
    )
    base_interval = 1000 / sampling_frequency

    def update_interval(event):
        # Ensure minimum interval to prevent browser freezing
        player.interval = max(10, int(event.new * base_interval))

    speed_dropdown.param.watch(update_interval, "value")
    return speed_dropdown


# --- Modified 2D Decoding Plot Function ---
def create_2D_decoding_plot(
    position_info,  # Pandas DataFrame with time index, x, y, head_direction cols
    posterior,  # xarray DataArray (time, y_pos, x_pos)
    map_estimate,  # Pandas Series/DataFrame with time index, x_position, y_position
    player,  # The pn.widgets.Player object
    position_names=("x", "y"),
    head_dir_name="head_direction",
    head_dir_radius=4,
    cmap=Viridis256,
    animal_color="magenta",
    map_color="lime",
    decode_opacity=0.8,  # Adjusted opacity slightly
    width=400,
    height=400,
):
    # Check if posterior is an xarray DataArray
    if not isinstance(posterior, xr.DataArray):
        raise TypeError("posterior must be an xarray.DataArray with dims (time, y, x)")

    x_dim, y_dim = posterior.dims[2], posterior.dims[1]  # Get spatial dim names

    def get_posterior_image(time_ind):
        # Select slice and ensure dimension order for hv.Image
        posterior_slice = posterior.isel(time=time_ind).transpose(y_dim, x_dim)
        # Use hv.Image - generally better for raster data
        return hv.Image(
            posterior_slice,
            kdims=[x_dim, y_dim],  # Use dimension names from xarray
            vdims=[posterior.name or "probability"],
        ).opts(
            cmap=cmap,
            alpha=decode_opacity,
            width=width,
            height=height,
            colorbar=True,
            tools=["hover"],
            axiswise=True,  # Axiswise helps zooming
            clim=(
                0,
                float(np.nanpercentile(posterior.values, 99.5)),
            ),  # Use float() for clim robustness
        )

    def get_animal_position_marker(time_ind):
        time_val = player.param.values_[
            time_ind
        ]  # Get actual time from player if needed
        # Use .loc with index.asof for robustness with float time index
        try:
            current_pos = position_info.loc[position_info.index.asof(time_val)]
        except KeyError:
            return hv.Points([]).opts(
                color=animal_color
            )  # Return empty if time not found

        head_point = hv.Points(
            (current_pos[position_names[0]], current_pos[position_names[1]])
        ).opts(size=10, color=animal_color)

        # Head direction line
        x_start, y_start = (
            current_pos[position_names[0]],
            current_pos[position_names[1]],
        )
        direction = current_pos[head_dir_name]
        x_end = x_start + head_dir_radius * np.cos(direction)
        y_end = y_start + head_dir_radius * np.sin(direction)
        head_direction_line = hv.Segments(
            [{"x0": x_start, "y0": y_start, "x1": x_end, "y1": y_end}]
        ).opts(color=animal_color, line_width=3)

        return head_point * head_direction_line

    def get_map_estimate_marker(time_ind):
        # Ensure map_estimate has time index compatible with player values
        time_val = player.param.values_[time_ind]
        try:
            map_pos = map_estimate.loc[map_estimate.index.asof(time_val)]
            return hv.Points((map_pos["x_position"], map_pos["y_position"])).opts(
                color=map_color, size=10, marker="+", line_width=2
            )
        except KeyError:
            return hv.Points([]).opts(color=map_color)

    # Create DynamicMaps linked to the player widget
    posterior_map = hv.DynamicMap(
        pn.bind(
            get_posterior_image, time_ind=player.param.value_throttled
        )  # Use throttled value
    )
    animal_marker = hv.DynamicMap(
        pn.bind(get_animal_position_marker, time_ind=player.param.value)
    )
    map_marker = hv.DynamicMap(
        pn.bind(get_map_estimate_marker, time_ind=player.param.value)
    )

    # Overlay components
    combined_plot = posterior_map * map_marker * animal_marker
    return combined_plot.opts(
        width=width, height=height, shared_axes=False, title="Decoded Posterior"
    )


# --- NEW: Function for Static Datashaded Overview Plot ---
def create_overview_plot(data_series, ylabel, time_vector, player, width, height):
    """Creates a datashaded overview plot with a VLine linked to the player."""
    df = pd.DataFrame({"time": time_vector, "value": data_series.values})

    line = df.hvplot(
        x="time",
        y="value",
        datashade=True,
        width=width,
        height=height,
        xlabel="Time (s)",
        ylabel=ylabel,
        grid=True,
    ).opts(
        shared_axes=False
    )  # Independent y-axis

    # Dynamic VLine linked to player value
    def get_vline(time_ind):
        return hv.VLine(time_vector[time_ind]).opts(color="red", line_width=1)

    vline = hv.DynamicMap(pn.bind(get_vline, time_ind=player.param.value))

    return line * vline  # Overlay line and vline


# --- NEW: Function for Dynamic Time Slice Plot ---
def create_time_slice_plot(
    data_series,
    ylabel,
    time_vector,
    player,
    window_seconds,
    sampling_frequency,
    width,
    height,
    ylim=None,
):
    """Creates a dynamically updating time slice plot."""
    n_time = len(time_vector)

    def get_slice(time_ind):
        selected_time = time_vector[time_ind]
        window_half_width_sec = window_seconds / 2.0
        start_time = selected_time - window_half_width_sec
        end_time = selected_time + window_half_width_sec

        # Slice the original Series/array using time
        # Ensure data_series index is comparable to start/end_time
        # If data_series index is not time, you need to find corresponding indices
        try:
            # Assuming data_series is indexed by time matching time_vector
            idxer = data_series.index.slice_indexer(start_time, end_time)
            window_data = data_series.iloc[idxer]
            if window_data.empty:
                raise ValueError("Empty slice")
            relative_time = window_data.index - selected_time
        except (AttributeError, ValueError, IndexError):
            # Fallback if index isn't time or slicing fails
            start_idx = max(
                0, time_ind - int(window_half_width_sec * sampling_frequency)
            )
            end_idx = min(
                n_time, time_ind + 1 + int(window_half_width_sec * sampling_frequency)
            )  # +1 to include current point
            window_vals = data_series.iloc[start_idx:end_idx].values.squeeze()
            relative_time = time_vector[start_idx:end_idx] - selected_time
            window_data = pd.Series(
                window_vals, index=relative_time
            )  # Create Series for hvplot

        curve = window_data.hvplot(
            x="index",
            y=0,  # hvplot needs explicit column if index isn't named time
            kind="line",
            color="black",
            xlabel="Time rel. to Current (s)",
            ylabel=ylabel,
            width=width,
            height=height,
            grid=True,
            xlim=(
                -window_half_width_sec,
                window_half_width_sec,
            ),  # Set fixed relative xlim
            ylim=ylim,
        ).opts(shared_axes=False)

        return curve * hv.VLine(0).opts(color="red")

    return hv.DynamicMap(
        pn.bind(get_slice, time_ind=player.param.value_throttled)
    )  # Use throttled for slice


# --- Main Visualization Function (Refactored) ---
def create_visualization_optimized(
    position_info,  # Pandas DataFrame (index=time, cols=[x, y, head_direction, speed])
    posterior,  # xarray DataArray (time, y, x) - Prob or LogProb
    multiunit_firing_rate,  # Pandas Series (index=time)
    non_local_prob,  # Pandas Series (index=time) - Prob of non-local state(s)
    map_estimate,  # Pandas DataFrame (index=time, cols=[x_position, y_position])
    environment,  # MockEnvironment or actual Environment object
    sampling_frequency=250,
    width=900,  # Adjusted width
    height=450,  # Adjusted height
    time_slice_window_sec=1.0,  # Default window for slice view
    position_names=("x", "y"),
    speed_name="speed",
    head_dir_name="head_direction",
):
    # Prepare common data
    time_vector = posterior["time"].values
    n_time = len(time_vector)

    # Ensure DataFrames/Series use the float time index for consistency if needed
    # Or ensure mapping works correctly in plotting functions
    # Example: Force float index if it's not already
    # position_info.index = time_vector
    # multiunit_firing_rate.index = time_vector
    # non_local_prob.index = time_vector
    # map_estimate.index = time_vector

    # Create Player and Speed Control
    player = create_player(
        n_time, sampling_frequency=sampling_frequency, width=int(width * 0.7)
    )
    speed_control = create_playback_speed_dropdown(
        player, sampling_frequency=sampling_frequency, width=int(width * 0.2)
    )

    # Create 2D Decoding Plot
    decode_plot_map = create_2D_decoding_plot(
        position_info,
        posterior,
        map_estimate,
        player,
        position_names=position_names,
        head_dir_name=head_dir_name,
        width=width // 2,
        height=height,
    )
    decode_controls = pn.Column(
        pn.Row(player, speed_control), sizing_mode="stretch_width"
    )
    decode_panel = pn.Column(decode_plot_map, decode_controls)

    # Create Time Series Plots
    ts_overview_height = 100
    ts_slice_height = 100
    ts_plot_width = width // 2 - 20  # Adjust for padding/margins

    # -- Overview Plots (Datashaded) --
    mufr_overview = create_overview_plot(
        multiunit_firing_rate,
        "MU Rate (Hz)",
        time_vector,
        player,
        ts_plot_width,
        ts_overview_height,
    )
    speed_overview = create_overview_plot(
        position_info[speed_name],
        "Speed (cm/s)",
        time_vector,
        player,
        ts_plot_width,
        ts_overview_height,
    )
    prob_overview = create_overview_plot(
        non_local_prob,
        "Non-Local Prob",
        time_vector,
        player,
        ts_plot_width,
        ts_overview_height,
    )

    # -- Time Slice Plots (Dynamic Raw Data) --
    # Widget to control window size
    window_slider = pn.widgets.FloatSlider(
        name="Time Slice Window (s)",
        start=0.2,
        end=5.0,
        step=0.1,
        value=time_slice_window_sec,
        width=ts_plot_width // 2,
    )

    mufr_slice = create_time_slice_plot(
        multiunit_firing_rate,
        "MU Rate (Hz)",
        time_vector,
        player,
        window_slider,
        sampling_frequency,
        ts_plot_width,
        ts_slice_height,
        ylim=(0, float(multiunit_firing_rate.quantile(0.999)) * 1.1),  # Example ylim
    )
    speed_slice = create_time_slice_plot(
        position_info[speed_name],
        "Speed (cm/s)",
        time_vector,
        player,
        window_slider,
        sampling_frequency,
        ts_plot_width,
        ts_slice_height,
        ylim=(
            0,
            float(position_info[speed_name].quantile(0.999)) * 1.1,
        ),  # Example ylim
    )
    prob_slice = create_time_slice_plot(
        non_local_prob,
        "Non-Local Prob",
        time_vector,
        player,
        window_slider,
        sampling_frequency,
        ts_plot_width,
        ts_slice_height,
        ylim=(-0.01, 1.01),
    )

    # --- Layout ---
    timeseries_panel = pn.Column(
        pn.Row(pn.pane.Markdown("#### Time Slice View"), window_slider),
        mufr_slice,
        speed_slice,
        prob_slice,
        pn.pane.Markdown("#### Full Overview"),
        mufr_overview,
        speed_overview,
        prob_overview,
        width=width // 2,  # Constrain width of the column
    )

    layout = pn.Row(decode_panel, timeseries_panel)

    return layout


In [28]:

(time_vector_data,
    spatial_posterior_da_data,
    animal_pos_df_data,
    timeseries_df_data,
    environment_data) = generate_fake_data(duration_minutes=1, time_bin_ms=4)

# Prepare inputs matching the function signature
pos_info = animal_pos_df_data.copy()
# Add speed and fake head direction if not present
if 'speed' not in pos_info: pos_info['speed'] = timeseries_df_data['Speed']
if 'head_direction' not in pos_info: pos_info['head_direction'] = np.arctan2(np.gradient(pos_info['y']), np.gradient(pos_info['x']))

posterior_xr = spatial_posterior_da_data # Already xarray
muf_rate = timeseries_df_data['Theta Power'] * 20 # Use theta power as fake multiunit
non_local_p = (spatial_posterior_da_data > 0.05).mean(dim=['x','y']) # Fake non-local prob based on spread

# Fake MAP estimate
map_estimate_df = pd.DataFrame({
        'x_position': spatial_posterior_da_data.mean(dim='y').argmax(dim='x').values * environment_data.place_bin_size, # very rough MAP x
        'y_position': spatial_posterior_da_data.mean(dim='x').argmax(dim='y').values * environment_data.place_bin_size  # very rough MAP y
    }, index=time_vector_data)


viz_layout = create_visualization_optimized(
    position_info=pos_info,
    posterior=posterior_xr,
    multiunit_firing_rate=muf_rate,
    non_local_prob=non_local_p,
    map_estimate=map_estimate_df,
    environment=environment_data,
    sampling_frequency=1.0 / (time_vector_data[1] - time_vector_data[0]),
    position_names=('x', 'y'),
    speed_name='speed',
    head_dir_name='head_direction'
)

# Display in Jupyter
viz_layout.servable()

Generating fake data...
Generated time vector: 15000 points, 60.0 s total, 0.0040 s step
Generated environment: 20 x 20 = 400 bins
Generated animal position data.
Generated auxiliary time series data.
Generating posterior 14000-15000/15000
Generated posterior probability data.
Created xarray DataArray.


Invoked as wrapped(time_ind=0)


AttributeError: 'Player.param' object has no attribute 'values_'

Row
    [0] Column
        [0] HoloViews(DynamicMap)
        [1] Column(sizing_mode='stretch_width')
            [0] Row
                [0] Player(end=14999, interval=4, name='Time Index', show_loop_controls=False, width=630)
                [1] Select(name='Playback Speed', options={'1/8x': 8, '1/4x': 4, ...}, value=1, width=180)
    [1] Column(width=450)
        [0] Row
            [0] Markdown(str)
            [1] FloatSlider(end=5.0, name='Time Slice Window (s)', start=0.2, value=1.0, width=215)
        [1] HoloViews(DynamicMap)
        [2] HoloViews(DynamicMap)
        [3] HoloViews(DynamicMap)
        [4] Markdown(str)
        [5] HoloViews(DynamicMap)
        [6] HoloViews(DynamicMap)
        [7] HoloViews(DynamicMap)

In [ ]:
import holoviews as hv
import panel as pn
import numpy as np
from non_local_detector.analysis.posterior import maximum_a_posteriori_estimate

from holoviews.operation.datashader import datashade, rasterize


def create_2D_decoding_plot(
    position_info,
    posterior,
    map_estimate=None,
    sampling_frequency=250,
    width=800,
    height=600,
    head_dir_radius=4,
    position_names=("x", "y"),
    head_dir_name="head_direction",
    cmap="viridis",
    animal_color="magenta",
    decode_opacity=1.0,
):
    n_time = len(posterior.time)

    if map_estimate is None:
        map_estimate = maximum_a_posteriori_estimate(acausal_posterior)

    def get_animal_position_dot(time_ind):
        head_point = hv.Scatter(
            position_info.iloc[[time_ind]],
            kdims=position_names[0],
            vdims=position_names[1],
        ).opts(size=10, color=animal_color, width=width, height=height)

        # Calculate the end point of the head direction line
        x_start = position_info.iloc[time_ind][position_names[0]]
        y_start = position_info.iloc[time_ind][position_names[1]]

        direction = position_info.iloc[time_ind][head_dir_name]

        x_end = x_start + head_dir_radius * np.cos(direction)
        y_end = y_start + head_dir_radius * np.sin(direction)

        head_direction_line = hv.Segments(
            [{"x0": x_start, "y0": y_start, "x1": x_end, "y1": y_end}]
        ).opts(color=animal_color, line_width=3)

        return head_point * head_direction_line

    def get_posterior_quadmesh(time_ind):
        posterior_slice = posterior.isel(time=time_ind)
        return hv.QuadMesh(
            posterior_slice,
            kdims=["x_position", "y_position"],
            vdims=["probability"],
        ).opts(cmap=cmap, alpha=decode_opacity, width=width, height=height)

    def get_most_likely_decoded_position(time_ind):
        return hv.Scatter(
            map_estimate[[time_ind]],
            kdims="x_position",
            vdims="y_position",
        ).opts(color="green", size=10, width=width, height=height)

    # Create player controls
    player = create_player(
        n_time, sampling_frequency=sampling_frequency, width=int(width * 0.7)
    )
    playback_speed_dropdown = create_playback_speed_dropdown(
        player, sampling_frequency=sampling_frequency, width=int(width * 0.2)
    )

    animal_position_dot = hv.DynamicMap(pn.bind(get_animal_position_dot, player))
    # posterior_image = datashade(hv.DynamicMap(pn.bind(get_posterior_quadmesh, player)))
    posterior_image = hv.DynamicMap(pn.bind(get_posterior_quadmesh, player))
    most_likely_decoded_position_dot = hv.DynamicMap(
        pn.bind(get_most_likely_decoded_position, player)
    )

    # Overlay the animal position on the posterior
    decode_plot = pn.Column(
        posterior_image * most_likely_decoded_position_dot * animal_position_dot,
        pn.Row(player, playback_speed_dropdown),
    )

    return decode_plot, player


def create_scrolling_line_plot(
    data,
    ylabel,
    ylim,
    player,
    window_lim=(-0.4, 0.4),
    height_factor=3,
    sampling_frequency=250,
    width=800,
    height=600,
):
    n_time = len(data)
    n_samples = int((window_lim[1] - window_lim[0]) * sampling_frequency)
    half_samples = n_samples // 2

    def plot_func(n):
        start_idx = max(0, n - half_samples)
        end_idx = min(n_time, n + half_samples)
        window = np.asarray(data, dtype=float)[start_idx:end_idx].squeeze()
        line = hv.Curve(
            (np.arange(start_idx - n, end_idx - n) / sampling_frequency, window),
            kdims="time relative to current (s)",
            vdims=ylabel,
        ).opts(
            color="black",
            width=width,
            height=int(height / height_factor),
            ylim=ylim,
            xlim=window_lim,
        )
        return line

    return hv.DynamicMap(pn.bind(plot_func, player))


def create_player(n_time, sampling_frequency=250, width=800):
    base_interval = int(1000 / sampling_frequency)  # ms/frame
    player = pn.widgets.Player(
        start=0,
        end=n_time - 1,
        value=0,
        step=1,
        interval=base_interval,
        name="Time",
        width=width,
        show_loop_controls=False,
    )
    return player


def create_playback_speed_dropdown(player, sampling_frequency=250, width=800):
    # Dropdown to control the speed of the player (interval)
    speed_options = {
        "1/8x": 8,
        "1/4x": 4,
        "1/2x": 2,
        "1x": 1,
        "2x": 0.5,
        "4x": 0.25,
        "8x": 0.125,
    }
    speed_dropdown = pn.widgets.Select(
        name="Playback Speed", options=speed_options, value=1, width=width
    )
    base_interval = 1000 / sampling_frequency  # ms/frame

    # Update the player's interval property based on the dropdown's value
    def update_interval(event):
        player.interval = int(event.new * base_interval)

    speed_dropdown.param.watch(update_interval, "value")

    return speed_dropdown


def create_visualization(
    position_info,
    posterior,
    multiunit_firing_rate,
    non_local_prob,
    map_estimate=None,
    sampling_frequency=250,
    width=800,
    height=600,
    window_lim=(-0.4, 0.4),
    position_names=("x", "y"),
    speed_name="speed",
    head_dir_name="head_direction",
):
    decode_plot, player = create_2D_decoding_plot(
        position_info,
        posterior,
        map_estimate=map_estimate,
        sampling_frequency=sampling_frequency,
        width=width // 2,
        height=height,
        position_names=position_names,
        head_dir_name=head_dir_name,
    )

    multiunit_plot = create_scrolling_line_plot(
        multiunit_firing_rate,
        "Rate (spikes/s)",
        (0, multiunit_firing_rate.to_numpy().max()),
        player,
        window_lim=window_lim,
        height_factor=3,
        sampling_frequency=sampling_frequency,
        width=width // 2,
        height=height,
    )
    speed_plot = create_scrolling_line_plot(
        position_info[speed_name],
        "Speed (cm/s)",
        (0, 100),
        player,
        window_lim=window_lim,
        height_factor=3,
        sampling_frequency=sampling_frequency,
        width=width // 2,
        height=height,
    )
    non_local_plot = create_scrolling_line_plot(
        non_local_prob,
        "Non-Local prob",
        (0.0, 1.01),
        player,
        window_lim=window_lim,
        height_factor=3,
        sampling_frequency=sampling_frequency,
        width=width // 2,
        height=height,
    )

    # Layout the components using Panel
    layout = pn.Row(
        decode_plot,
        pn.Column(multiunit_plot, speed_plot, non_local_plot),
    )

    return layout


Generating fake data...
Generated time vector: 15000 points, 60.0 s total, 0.0040 s step
Generated environment: 20 x 20 = 400 bins
Generated animal position data.
Generated auxiliary time series data.
Generating posterior 14000-15000/15000
Generated posterior probability data.
Created xarray DataArray.


KeyError: "None of [Index([0], dtype='int64')] are in the [columns]"

Row
    [0] Column
        [0] HoloViews(DynamicMap)
        [1] Row
            [0] Player(end=14999, interval=4, name='Time', show_loop_controls=False, width=280)
            [1] Select(name='Playback Speed', options={'1/8x': 8, '1/4x': 4, ...}, value=1, width=80)
    [1] Column
        [0] HoloViews(DynamicMap)
        [1] HoloViews(DynamicMap)
        [2] HoloViews(DynamicMap)

In [32]:
posterior_xr

<xarray.DataArray 'Posterior Probability' (time: 15000, y: 20, x: 20)> Size: 48MB
array([[[6.24755646e-03, 7.05456221e-03, 7.65346615e-03, ...,
         2.13828765e-04, 1.22322334e-04, 6.72316251e-05],
        [7.04114970e-03, 7.95066501e-03, 8.62564448e-03, ...,
         2.40990275e-04, 1.37860277e-04, 7.57716941e-05],
        [7.62439146e-03, 8.60924494e-03, 9.34013519e-03, ...,
         2.60952298e-04, 1.49279700e-04, 8.20481146e-05],
        ...,
        [2.07021668e-04, 2.33762951e-04, 2.53608485e-04, ...,
         7.08552022e-06, 4.05332448e-06, 2.22781551e-06],
        [1.18203121e-04, 1.33471586e-04, 1.44802787e-04, ...,
         4.04561809e-06, 2.31432588e-06, 1.27201538e-06],
        [6.48440757e-05, 7.32200771e-05, 7.94361671e-05, ...,
         2.21935228e-06, 1.26959696e-06, 6.97804432e-07]],

       [[2.86503230e-13, 3.45932693e-12, 3.25297011e-11, ...,
         1.20996438e-09, 2.08392998e-10, 2.79524603e-11],
        [2.91925948e-12, 3.52480247e-11, 3.31453988e-10, ...,
         1.23286567e-08, 2.12337304e-09, 2.84815234e-10],
        [2.31655307e-11, 2.79707646e-10, 2.63022441e-09, ...,
         9.78329876e-08, 1.68498428e-08, 2.26012662e-09],
...
        [8.58864997e-39, 9.92712962e-37, 8.93611730e-35, ...,
         1.72660078e-18, 2.84668343e-18, 3.65521388e-18],
        [9.54112828e-41, 1.10280449e-38, 9.92712962e-37, ...,
         1.91808021e-20, 3.16237964e-20, 4.06057583e-20],
        [8.25469337e-43, 9.54112828e-41, 8.58864997e-39, ...,
         1.65946453e-22, 2.73599447e-22, 3.51308644e-22]],

       [[3.51308644e-22, 4.06057583e-20, 3.65521388e-18, ...,
         7.06245779e-02, 1.16440244e-01, 1.49512233e-01],
        [2.73599447e-22, 3.16237964e-20, 2.84668343e-18, ...,
         5.50024766e-02, 9.06837530e-02, 1.16440244e-01],
        [1.65946453e-22, 1.91808021e-20, 1.72660078e-18, ...,
         3.33606884e-02, 5.50024766e-02, 7.06245779e-02],
        ...,
        [8.58864997e-39, 9.92712962e-37, 8.93611730e-35, ...,
         1.72660078e-18, 2.84668343e-18, 3.65521388e-18],
        [9.54112828e-41, 1.10280449e-38, 9.92712962e-37, ...,
         1.91808021e-20, 3.16237964e-20, 4.06057583e-20],
        [8.25469337e-43, 9.54112828e-41, 8.58864997e-39, ...,
         1.65946453e-22, 2.73599447e-22, 3.51308644e-22]]])
Coordinates:
  * time     (time) float64 120kB 0.0 0.004 0.008 0.012 ... 59.99 59.99 60.0
  * y        (y) float64 160B 2.5 7.5 12.5 17.5 22.5 ... 82.5 87.5 92.5 97.5
  * x        (x) float64 160B 2.5 7.5 12.5 17.5 22.5 ... 82.5 87.5 92.5 97.5

In [33]:
pos_info

,x,y,speed,head_direction
0.000,54.821417,51.426493,2.995285,0.895543
0.004,54.821590,51.426709,4.240319,0.894586
0.008,54.821936,51.427140,3.471137,0.894599
0.012,54.822457,51.427789,2.736160,0.894391
0.016,54.823152,51.428655,3.205348,0.893770
...,...,...,...,...
59.980,100.000000,0.000000,0.592413,0.000000
59.984,100.000000,0.000000,0.555184,0.000000
59.988,100.000000,0.000000,1.673073,0.000000
59.992,100.000000,0.000000,0.631375,0.000000


In [34]:
muf_rate

0.000     31.403599
0.004     31.403798
0.008     31.404200
0.012     31.404802
0.016     31.405599
            ...    
59.980    17.919928
59.984    17.919908
59.988    17.919893
59.992    17.919888
59.996    17.919882
Name: Theta Power, Length: 15000, dtype: float64

In [35]:
map_estimate_df

,x_position,y_position
0.000,20.0,15.0
0.004,50.0,50.0
0.008,50.0,50.0
0.012,50.0,50.0
0.016,50.0,50.0
...,...,...
59.980,95.0,0.0
59.984,95.0,0.0
59.988,95.0,0.0
59.992,95.0,0.0
